In [2]:
import pandas as pd
import numpy as np
import os
import datetime

In [13]:
def load_finance_data(filepath="financeData_target_variables.csv"):
    """
    Lädt die Finanzdaten aus der CSV-Datei in ein DataFrame.
    """
    df = pd.read_csv(filepath, parse_dates=["Date"])
    return df

In [14]:
df = load_finance_data()
df.head()

,Date,sp500_close,bitcoin_close,nasdaq_close,sp500_volume,bitcoin_volume,nasdaq_volume
0,2014-09-17,2001.569946,457.334015,4562.189941,3.209420e+09,21056800,1.796710e+09
1,2014-09-18,2011.359985,424.440002,4593.430176,3.235340e+09,34483200,1.774840e+09
2,2014-09-19,2010.400024,394.795990,4579.790039,4.880220e+09,37919700,3.178490e+09
3,2014-09-20,NaN,408.903992,NaN,NaN,36863600,NaN
4,2014-09-21,NaN,398.821014,NaN,NaN,26580100,NaN


In [24]:
def update_csv_with_btc_change(filepath="financeData_target_variables.csv"):
    # CSV laden
    df = pd.read_csv(filepath, parse_dates=["Date"])

    # Nach Datum sortieren (für korrekte pct_change)
    df = df.sort_values("Date").reset_index(drop=True)

    # Prozentuale Änderung berechnen
    df["btc_change"] = df["bitcoin_close"].pct_change() * 100
    df["sp500_change"] = df["sp500_close"].pct_change() * 100
    df["nasdaq_change"] = df["nasdaq_close"].pct_change() * 100
    

    df.to_csv(filepath, index=False)
    
update_csv_with_btc_change()

C:\Users\malte\AppData\Local\Temp\ipykernel_19268\766523630.py:10: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["sp500_change"] = df["sp500_close"].pct_change() * 100
C:\Users\malte\AppData\Local\Temp\ipykernel_19268\766523630.py:11: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["nasdaq_change"] = df["nasdaq_close"].pct_change() * 100


In [26]:
def update_csv_with_btc_change(filepath="financeData_target_variables.csv"):
    df = pd.read_csv(filepath, parse_dates=["Date"])
    df = df.sort_values("Date").reset_index(drop=True)

    # Bitcoin: täglich vorhanden → shift(1) reicht
    df["btc_change"] = df["bitcoin_close"].div(df["bitcoin_close"].shift(1)).subtract(1).mul(100)

    # SP500 / NASDAQ: evtl. NaNs → erst forward fill, dann shift
    df["sp500_last_valid"] = df["sp500_close"].ffill().shift(1)
    df["sp500_change"] = df["sp500_close"].div(df["sp500_last_valid"]).subtract(1).mul(100)

    df["nasdaq_last_valid"] = df["nasdaq_close"].ffill().shift(1)
    df["nasdaq_change"] = df["nasdaq_close"].div(df["nasdaq_last_valid"]).subtract(1).mul(100)

    # Hilfsspalten entfernen
    df = df.drop(columns=["sp500_last_valid", "nasdaq_last_valid"])
    
    #df.head()
    #return df
    df.to_csv(filepath, index=False)

update_csv_with_btc_change()
